In [53]:
# 1. Importa las librerías requeridas.
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# 2. Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition.
EmpleadosAttrition = pd.read_csv("empleadosRETO.csv")

# 3. Elimina las columnas que, con alta probabilidad (estimada por ti), no tienen relación alguna con la salida. Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado, tal es caso de las siguientes:
# EmployeeCount: número de empleados, todos tienen un 1
# EmployeeNumber: ID del empleado, el cual es único para cada empleado
# Over18: mayores de edad, todos dicen “Y”
# StandardHours: horas de trabajo, todos tienen “80”
EmpleadosAttrition.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis = 1, inplace=True)

# 4. Analiza la información proporcionada, si detectaste que no se cuenta con los años que el empelado lleva en la compañía y parece ser un buen dato. Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’.
# Se convierte la columna 'HiringDate' a formato datetime
EmpleadosAttrition['HiringDate'] = pd.to_datetime(EmpleadosAttrition['HiringDate'], format='%m/%d/%Y' , errors='coerce')

# Verificar si hay fechas no válidas
invalid_dates = EmpleadosAttrition[EmpleadosAttrition['HiringDate'].isna()]
if not invalid_dates.empty:
    # El registro 36,Travel_Rarely,Sales,11 km,4,Marketing,1,2045,2,Female,2,2,Sales Executive,4,Married,6652,4,2/30/2012,Y,No,13,3,1,80,8,2,2,3,0,No tiene fecha del 30 de febrero
    # que genera un error al convertirlo a fecha
    #print("Hay fechas no válidas que eliminam:" + str(invalid_dates))
    EmpleadosAttrition = EmpleadosAttrition.dropna(subset=['HiringDate'])

# 5. Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. No se te olvide que debe ser un entero.
# Se crea la nueva columna 'Year' con el año de contratación
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].dt.year

# 6. Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018. Para su cálculo, usa la variable Year que acabas de crear.
# Se crea la columna 'YearsAtCompany' calculando los años hasta 2018
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']

# 7. La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera
# 8. Renombra la variable DistanceFromHome a DistanceFromHome_km
EmpleadosAttrition.rename(columns={'DistanceFromHome': 'DistanceFromHome_km'}, inplace=True)

# 9. Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.extract(r'(\d+)').astype(int)

# 10. Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles
EmpleadosAttrition.drop(columns=['Year', 'HiringDate', 'DistanceFromHome_km'], inplace=True)


# Mostrar los primeros registros del DataFrame
EmpleadosAttrition.head()

,Age,BusinessTravel,Department,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,...,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition,YearsAtCompany,DistanceFromHome
0,50,Travel_Rarely,Research & Development,2,Medical,4,Male,3,4,Research Director,...,4,3,32,1,2,4,1,No,5,1
1,36,Travel_Rarely,Research & Development,2,Medical,2,Male,3,2,Manufacturing Director,...,4,4,7,0,3,2,0,No,3,6
2,21,Travel_Rarely,Sales,1,Marketing,2,Male,3,1,Sales Representative,...,3,2,1,3,3,0,1,Yes,1,7
3,52,Travel_Rarely,Research & Development,4,Life Sciences,2,Male,3,3,Healthcare Representative,...,3,4,18,4,3,6,4,No,8,7
4,33,Travel_Rarely,Research & Development,1,Medical,2,Male,3,3,Manager,...,3,4,15,2,4,6,7,Yes,7,15


In [54]:
# 11. Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar. Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados y colócalo en una variable llamada SueldoPromedio. Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.
# Se crea un nuevo DataFrame con el ingreso mensual promedio por departamento
SueldoPromedioDepto = EmpleadosAttrition.groupby('Department')['MonthlyIncome'].mean().reset_index()

# Se renombra la columna de ingreso promedio
SueldoPromedioDepto.rename(columns={'MonthlyIncome': 'SueldoPromedio'}, inplace=True)

# Imprime el nuevo dataframe
SueldoPromedioDepto.head()


,Department,SueldoPromedio
0,Human Resources,6239.888889
1,Research & Development,6804.149813
2,Sales,7192.609756


In [64]:
# 12. La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables. Escala dicha variable para que tenga un valor entre 0 y 1.
# Se crea una instancia de MinMaxScaler
scaler = MinMaxScaler()

# Se ajusta y transformar la columna 'MonthlyIncome'
EmpleadosAttrition['MonthlyIncome'] = scaler.fit_transform(EmpleadosAttrition[['MonthlyIncome']])

# 13. Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida. Convierte todas las variables categóricas que quedan a numéricas:
#a) BusinessTravel
#b) Department
#c) EducationField
#d) Gender
#e) JobRole
#f) MaritalStatus
#g) Attrition

# Columnas categóricas
categorical_columns = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Attrition']

# Se aplica LabelEncoder a cada columna categórica
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    EmpleadosAttrition[col] = le.fit_transform(EmpleadosAttrition[col])

EmpleadosAttrition.head()


,Age,BusinessTravel,Department,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,...,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition,YearsAtCompany,DistanceFromHome
0,50,2,1,2,3,4,1,3,4,5,...,4,3,32,1,2,4,1,0,5,1
1,36,2,1,2,3,2,1,3,2,4,...,4,4,7,0,3,2,0,0,3,6
2,21,2,2,1,2,2,1,3,1,8,...,3,2,1,3,3,0,1,1,1,7
3,52,2,1,4,1,2,1,3,3,0,...,3,4,18,4,3,6,4,0,8,7
4,33,2,1,1,3,2,1,3,3,3,...,3,4,15,2,4,6,7,1,7,15


In [65]:
# 14. Ahora debes hacer la evaluación de las variables para quedarte con las mejores. Calcula la correlación lineal de cada una de las variables con respecto al Attrition.
# Se calcula la correlación de todas las variables numéricas con respecto a 'Attrition'
# Se seleccionan solo las columnas numéricas para el cálculo de la correlación
numeric_columns = EmpleadosAttrition.select_dtypes(include=['number']).columns

# Se calcula la correlación solo para las columnas numéricas
correlations = EmpleadosAttrition[numeric_columns].corr()

# 15. Selecciona solo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame llamado EmpleadosAttritionFinal. No olvides mantener la variable de salidaAttrition; esto es equivalente a borrar las que no cumplen con el límite.
# Filtrar las variables con correlación >= 0.1 con Attrition
selected_columns = correlations.index[correlations['Attrition'].abs() >= 0.1].tolist()

# Crear el nuevo DataFrame con las variables seleccionadas
EmpleadosAttritionFinal = EmpleadosAttrition[selected_columns]

EmpleadosAttritionFinal.head()

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MaritalStatus,MonthlyIncome,TotalWorkingYears,YearsInCurrentRole,Attrition,YearsAtCompany
0,50,4,3,4,4,0,0.864269,32,4,0,5
1,36,2,3,2,2,0,0.207340,7,2,0,3
2,21,2,3,1,2,2,0.088062,1,0,1,1
3,52,2,3,3,2,2,0.497574,18,6,0,8
4,33,2,3,3,3,1,0.664470,15,6,1,7


In [66]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 16. Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal. Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna, por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).
# Se separan las características y la variable objetivo
X = EmpleadosAttritionFinal.drop(columns=['Attrition'])  # Variables predictoras
y = EmpleadosAttritionFinal['Attrition']                # Variable objetivo

# Se escalan las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Se apica PCA
pca = PCA()
EmpleadosAttritionPCA = pca.fit_transform(X_scaled)

# Se verifican las primeras filas del resultado
print("Componentes principales (primeras filas):")
EmpleadosAttritionPCA[:,0]

Componentes principales (primeras filas):


array([ 3.18596383e+00, -1.07092114e+00, -2.98696580e+00,  1.64725681e+00,
        1.10371306e+00,  3.50352423e+00, -1.87180104e+00, -6.71625583e-01,
        1.76263785e+00, -1.20570586e+00, -1.82868052e+00,  1.42180791e+00,
       -2.07841581e+00,  1.13295682e+00,  3.56067975e+00,  4.88507728e-01,
       -7.22164319e-01,  1.12939052e+00,  4.13310317e+00,  4.04299626e+00,
       -3.77983965e-01, -7.59455673e-01, -6.88764767e-01, -3.08960591e+00,
       -5.83282833e-01, -5.97984385e-01, -3.21408904e+00, -1.72657238e+00,
        4.12975097e+00, -6.91479808e-01, -7.57482713e-01, -1.31635695e+00,
        4.83713385e+00, -9.49405010e-03, -2.08719320e+00,  2.80902843e-01,
        1.39516077e-01,  3.25003293e+00, -3.42536589e-01,  5.27448984e-01,
        1.38808172e+00, -9.03076934e-01,  5.62361216e-01, -8.18661382e-01,
       -1.28609913e-01,  3.54070746e+00, -8.33664224e-01,  1.13944355e+00,
        6.25189914e-01, -1.83231955e+00,  8.32406069e-02,  3.48642065e+00,
        4.33858095e+00, -

In [67]:
# 17. Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza, al frame EmpleadosAttritionFinal. Puedes usar la instrucción assign, columna por columna, llamando a cada unaC0, C1, etc., hasta las que vayas a agregar.
# Se determina el número mínimo de componentes que explican al menos el 80% de la varianza
varianza_acumulada = pca.explained_variance_ratio_.cumsum()
n_componentes = (varianza_acumulada >= 0.8).argmax() + 1

# Se crea un DataFrame con los componentes principales seleccionados
componentes_seleccionados = pd.DataFrame(
    EmpleadosAttritionPCA[:, :n_componentes],
    columns=[f"C{i}" for i in range(n_componentes)]
)

# Se Agregan los componentes principales al DataFrame original
EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**componentes_seleccionados)


In [69]:
# 18. Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV llamado EmpleadosAttritionFinal.csv. Las últimas columnas que colocaste quedarán después de la variable Attrition, lo cual no importa, pero si gustas lo puedes arreglar antes de escribir el archivo.
# Guardar el DataFrame en un archivo CSV
EmpleadosAttritionFinal.to_csv("EmpleadosAttritionFinal.csv", index=False)

print("El archivo 'EmpleadosAttritionFinal.csv' ha sido guardado con éxito.")

El archivo 'EmpleadosAttritionFinal.csv' ha sido guardado con éxito.
